In [ ]:
!pip install emoji

In [ ]:
import emoji

 <font size=5.5>  **Heart disease** 💔   </font>

We want to use deep learning to predict if a person will have a heart disease.

 <font size=4>**Getting the data and importing the libraries**</font>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/My Drive/')

In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
data=pd.read_csv('https://raw.githubusercontent.com/pablo14/datasets/main/heart_disease/heart_disease.csv')
data

,age,gender,chest_pain,resting_blood_pressure,serum_cholestoral,fasting_blood_sugar,resting_electro,max_heart_rate,exer_angina,oldpeak,slope,num_vessels_flour,thal,heart_disease_severity,exter_angina,has_heart_disease
0,63,male,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0,0,no
1,67,male,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2,1,yes
2,67,male,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1,1,yes
3,37,male,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0,0,no
4,41,female,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0,0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45,male,1,110,264,0,0,132,0,1.2,2,0.0,7.0,1,0,yes
299,68,male,4,144,193,1,0,141,0,3.4,2,2.0,7.0,2,0,yes
300,57,male,4,130,131,0,0,115,1,1.2,2,1.0,7.0,3,1,yes
301,57,female,2,130,236,0,2,174,0,0.0,2,1.0,3.0,1,0,yes


 <font size=4>**Cleaning data**</font>

In [5]:
!pip install funpymodeling

     |████████████████████████████████| 225kB 4.3MB/s 
  Found existing installation: seaborn 0.11.0
    Uninstalling seaborn-0.11.0:
      Successfully uninstalled seaborn-0.11.0


In [6]:
from funpymodeling.exploratory import status, freq_tbl, profiling_num
from funpymodeling.data_prep import todf

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [7]:
#Viewing the main characteristics of the data
status(data)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,age,0,0.000000,0,0.000000,41,int64
1,gender,0,0.000000,0,0.000000,2,object
2,chest_pain,0,0.000000,0,0.000000,4,int64
3,resting_blood_pressure,0,0.000000,0,0.000000,50,int64
4,serum_cholestoral,0,0.000000,0,0.000000,152,int64
5,fasting_blood_sugar,0,0.000000,258,0.851485,2,int64
6,resting_electro,0,0.000000,151,0.498350,3,int64
7,max_heart_rate,0,0.000000,0,0.000000,91,int64
8,exer_angina,0,0.000000,204,0.673267,2,int64
9,oldpeak,0,0.000000,99,0.326733,40,float64


In [8]:
# Deleting NaN (In this case there's no impact)
data=data.dropna()

In [9]:
# Checking again the data
status(data)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,age,0,0.0,0,0.000000,41,int64
1,gender,0,0.0,0,0.000000,2,object
2,chest_pain,0,0.0,0,0.000000,4,int64
3,resting_blood_pressure,0,0.0,0,0.000000,50,int64
4,serum_cholestoral,0,0.0,0,0.000000,152,int64
5,fasting_blood_sugar,0,0.0,254,0.855219,2,int64
6,resting_electro,0,0.0,147,0.494949,3,int64
7,max_heart_rate,0,0.0,0,0.000000,91,int64
8,exer_angina,0,0.0,200,0.673401,2,int64
9,oldpeak,0,0.0,96,0.323232,40,float64


In [12]:
from keras.utils import to_categorical

# Getting only the columns we want to use
data['target'] = data['has_heart_disease'].apply(lambda x: 1 if x == 'yes' else 0)

data_x=data.drop(['target','heart_disease_severity','gender','has_heart_disease'], axis=1)

data_y=to_categorical(data['target'], num_classes=2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [13]:
# Normalizing the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_x = scaler.fit_transform(data_x)

 <font size=4>**Getting Test and Train sets**</font>

In [14]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2)

In [15]:
x_train.shape

(237, 13)

In [16]:
y_train.shape

(237, 2)

Model 1

In [18]:
from keras.models import Sequential
from keras.layers import Dense

In [20]:
_,in_shape=x_train.shape

In [21]:
#Creating the model
model1 = Sequential()

model1.add(Dense(256, input_dim=in_shape)) 
model1.add(Dense(128, activation='relu'))
model1.add(Dense(64, activation='relu'))
model1.add(Dense(2, activation='softmax')) 

In [23]:
#Compiling the model
model1.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005),

              loss='binary_crossentropy',

              metrics=['accuracy'])

In [24]:
#Looking at the model
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               3584      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 130       
Total params: 44,866
Trainable params: 44,866
Non-trainable params: 0
_________________________________________________________________


In [26]:
#Fitting the model
history1 = model1.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=40, batch_size=32)

Epoch 1/40
8/8 [==============================] - 1s 66ms/step - loss: 0.6898 - accuracy: 0.4717 - val_loss: 0.6430 - val_accuracy: 0.6500
Epoch 2/40
8/8 [==============================] - 0s 8ms/step - loss: 0.6390 - accuracy: 0.7089 - val_loss: 0.5965 - val_accuracy: 0.7833
Epoch 3/40
8/8 [==============================] - 0s 7ms/step - loss: 0.5986 - accuracy: 0.7665 - val_loss: 0.5447 - val_accuracy: 0.8167
Epoch 4/40
8/8 [==============================] - 0s 7ms/step - loss: 0.5570 - accuracy: 0.7626 - val_loss: 0.4912 - val_accuracy: 0.8500
Epoch 5/40
8/8 [==============================] - 0s 8ms/step - loss: 0.5025 - accuracy: 0.8255 - val_loss: 0.4401 - val_accuracy: 0.8667
Epoch 6/40
8/8 [==============================] - 0s 8ms/step - loss: 0.4665 - accuracy: 0.8051 - val_loss: 0.3959 - val_accuracy: 0.8833
Epoch 7/40
8/8 [==============================] - 0s 8ms/step - loss: 0.4210 - accuracy: 0.8430 - val_loss: 0.3747 - val_accuracy: 0.9000
Epoch 8/40
8/8 [=================

In [28]:
#Checking the results
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def plot_all(history):
  fig1 = go.Figure()
  fig1.add_trace(go.Scattergl(y=history.history['accuracy'], name='Train'))
  fig1.add_trace(go.Scattergl(y=history.history['val_accuracy'], name='Valid'))
  fig1.update_layout(height=300, width=400,xaxis_title='Epoch',yaxis_title='accuracy')

  fig2 = go.Figure()
  fig2.add_trace(go.Scattergl(y=history.history['loss'], name='Train'))
  fig2.add_trace(go.Scattergl(y=history.history['val_loss'], name='Valid'))
  fig2.update_layout(height=300, width=400,xaxis_title='Epoch', yaxis_title='Loss')

  return [fig1,fig2]

In [29]:
fig1,fig2=plot_all(history1)
fig1.show()
fig2.show()

It seems to be a good idea to train only until the 10th epoch.

Model 2

In [35]:
#Creating the model
model2 = Sequential()

model2.add(Dense(256, input_dim=in_shape)) 
model2.add(Dense(128, activation='relu'))
model2.add(Dense(64, activation='relu'))
model2.add(Dense(2, activation='softmax')) 

In [36]:
#Compiling the model
model2.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005),

              loss='binary_crossentropy',

              metrics=['accuracy'])

In [37]:
#Looking at the model
model2.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 256)               3584      
_________________________________________________________________
dense_9 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 130       
Total params: 44,866
Trainable params: 44,866
Non-trainable params: 0
_________________________________________________________________


In [38]:
#Fitting the model
history2 = model2.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=32)

Epoch 1/10
8/8 [==============================] - 1s 29ms/step - loss: 0.6773 - accuracy: 0.6801 - val_loss: 0.6199 - val_accuracy: 0.7667
Epoch 2/10
8/8 [==============================] - 0s 8ms/step - loss: 0.6164 - accuracy: 0.7963 - val_loss: 0.5729 - val_accuracy: 0.8333
Epoch 3/10
8/8 [==============================] - 0s 8ms/step - loss: 0.5727 - accuracy: 0.7994 - val_loss: 0.5182 - val_accuracy: 0.8667
Epoch 4/10
8/8 [==============================] - 0s 8ms/step - loss: 0.5109 - accuracy: 0.8374 - val_loss: 0.4670 - val_accuracy: 0.8833
Epoch 5/10
8/8 [==============================] - 0s 8ms/step - loss: 0.4788 - accuracy: 0.8338 - val_loss: 0.4245 - val_accuracy: 0.8833
Epoch 6/10
8/8 [==============================] - 0s 7ms/step - loss: 0.4628 - accuracy: 0.8168 - val_loss: 0.3984 - val_accuracy: 0.8500
Epoch 7/10
8/8 [==============================] - 0s 8ms/step - loss: 0.3892 - accuracy: 0.8451 - val_loss: 0.3857 - val_accuracy: 0.8833
Epoch 8/10
8/8 [=================

In [39]:
#Checking the results
fig1,fig2=plot_all(history2)
fig1.show()
fig2.show()

Model 2 has better accuracy than Model 1 so it seems to be better.